In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,labuan,5.2767,115.2417,32.20,62,20,1.54,MY,1700541943
1,1,waitangi,-43.9535,-176.5597,14.90,86,40,1.79,NZ,1700541943
2,2,margaret river,-33.9500,115.0667,27.61,37,35,4.84,AU,1700541944
3,3,yanchep,-31.5500,115.6833,35.63,24,0,4.67,AU,1700541944
4,4,utrik,11.2278,169.8474,27.60,76,100,10.16,MH,1700541944


# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city. 

In [3]:
Humidity = city_data_df["Humidity"].astype(int)

# Configure the map
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.80
)
    
# Display the map
city_plot


C:\Users\gloam\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

# Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 50 degrees, but higher than 0 degrees
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 50) & (city_data_df["Max Temp"] > 0)]

# Wind speeds less than 3mph
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 3)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,labuan,5.2767,115.2417,32.20,62,20,1.54,MY,1700541943
1,1,waitangi,-43.9535,-176.5597,14.90,86,40,1.79,NZ,1700541943
9,9,champerico,14.3000,-91.9167,25.98,70,24,0.61,GT,1700541946
16,16,puerto natales,-51.7236,-72.4875,3.16,92,100,2.74,CL,1700541947
19,19,lagarto,-10.9172,-37.6500,21.81,98,20,1.45,BR,1700541948


# Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\gloam\AppData\Local\Temp\ipykernel_18904\3768208391.py:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\gloam\AppData\Local\Temp\ipykernel_18904\3768208391.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,labuan,MY,5.2767,115.2417,62,
1,waitangi,NZ,-43.9535,-176.5597,86,
9,champerico,GT,14.3000,-91.9167,70,
16,puerto natales,CL,-51.7236,-72.4875,92,
19,lagarto,BR,-10.9172,-37.6500,98,
...,...,...,...,...,...,...
562,stefan voda,MD,46.5153,29.6631,87,
564,enoch,US,37.7733,-113.0244,83,
565,sake,CD,-1.5736,29.0450,89,
566,lambari,BR,-21.9756,-45.3503,95,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
labuan - nearest hotel: Labuan Avenue Hotel
waitangi - nearest hotel: Hotel Chathams
champerico - nearest hotel: Hotel y Restaurante El Submarino
puerto natales - nearest hotel: Dorotea Patagonia Hostel
lagarto - nearest hotel: Pousada Santa Cruz
nola - nearest hotel: Pamaran
toamasina - nearest hotel: Androranga
papatowai - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
tequisquiapan - nearest hotel: la Querencia
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
tura - nearest hotel: No hotel found
albertina - nearest hotel: Hotel Parque das Primaveras
trinidad - nearest hotel: Hotel Palace
mar del plata - nearest hotel: Gran hotel Mónaco
invercargill - nearest hotel: The Grand
sutherlin - nearest hotel: Best Western Plus Hartford Lodge
lahad datu - nearest hotel: Fajar Hotel Lahad Datu
chota - nearest hotel: No hotel found
winston - nearest hotel: Hilton Garden Inn Lakeland
urucurituba -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,labuan,MY,5.2767,115.2417,62,Labuan Avenue Hotel
1,waitangi,NZ,-43.9535,-176.5597,86,Hotel Chathams
9,champerico,GT,14.3000,-91.9167,70,Hotel y Restaurante El Submarino
16,puerto natales,CL,-51.7236,-72.4875,92,Dorotea Patagonia Hostel
19,lagarto,BR,-10.9172,-37.6500,98,Pousada Santa Cruz
...,...,...,...,...,...,...
562,stefan voda,MD,46.5153,29.6631,87,No hotel found
564,enoch,US,37.7733,-113.0244,83,Travelodge by Wyndham Cedar City
565,sake,CD,-1.5736,29.0450,89,No hotel found
566,lambari,BR,-21.9756,-45.3503,95,Parque Hotel


# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
    )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)